In [455]:
import numpy as np
import pandas as pd

data11 = pd.read_csv('../data/output/all_features_recursive_less_28_submission_v1.csv')
data22 = pd.read_csv('../data/output/output_all_features_onetime_add2ids_recursive_less_28_submission_v1.csv')
data33 = pd.read_csv('../data/output/categorical-emb.csv')
data33.iloc[30490:, 1:] = data33.iloc[:30490, 1:].values
data33.fillna(0, inplace=True)
data44 = pd.read_csv('../data/output/output_all_features_28_submission_v1.csv')
data55 = pd.read_csv('../data/output/output_cat_emb_baseline_28_submission_v1.csv')
#以下は過学習の可能性が非常に高い。

In [456]:
data1 = pd.read_csv('../data/output/output_all_features_recursive_less_1969_submission_v1.csv')
data2 = pd.read_csv('../data/output/output_all_features_onetime_add2ids_recursive_less_1969_submission_v1.csv')
data3 = pd.read_csv('../data/output/dnn_fake_eval_day_to_day.csv')
data4 = pd.read_csv('../data/output/output_all_features_1969_submission_v1.csv')
data5 = pd.read_csv('../data/output/output_cat_emb_baseline_1969_submission_v1.csv')

In [457]:
sales = pd.read_csv('../data/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
data_true = pd.concat([sales['id'],sales.iloc[:, -28:]], axis=1)
data_true.head()

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [458]:
data_true = pd.melt(data_true, id_vars=['id'])

In [459]:
train_data = pd.DataFrame()
for i, data in enumerate([data11, data22, data33, data44, data55]):
    data = data[data['id'].str.endswith('evaluation')]
    data = pd.melt(data, id_vars=['id'])
    if i == 0:
        train_data = data
    else:
        train_data = pd.concat([train_data, data['value']], axis=1)
    train_data.rename(columns={'value': i}, inplace=True)
train_data.head()

,id,variable,0,1,2,3,4
0,HOBBIES_1_001_CA_1_evaluation,F1,0.919392,0.805027,0.8296,0.783889,0.674006
1,HOBBIES_1_002_CA_1_evaluation,F1,0.131573,0.159024,0.2030,0.143639,0.206654
2,HOBBIES_1_003_CA_1_evaluation,F1,0.403828,0.356475,0.4678,0.414457,0.442239
3,HOBBIES_1_004_CA_1_evaluation,F1,1.671344,1.761921,2.2030,1.658047,1.634868
4,HOBBIES_1_005_CA_1_evaluation,F1,1.028402,1.066871,1.0020,0.871253,0.977911


In [460]:
test_data = pd.DataFrame()
for i, data in enumerate([data1, data2, data3, data4, data5]):
    data = data[data['id'].str.endswith('evaluation')]
    data = pd.melt(data, id_vars=['id'])
    if i == 0:
        test_data = data
    else:
        test_data = pd.concat([test_data, data['value']], axis=1)
    test_data.rename(columns={'value': i}, inplace=True)
test_data.head()

,id,variable,0,1,2,3,4
0,HOBBIES_1_001_CA_1_evaluation,F1,0.871367,0.889250,0.5215,0.890251,0.741056
1,HOBBIES_1_002_CA_1_evaluation,F1,0.200517,0.210504,0.1350,0.210739,0.215533
2,HOBBIES_1_003_CA_1_evaluation,F1,0.540492,0.485432,0.2546,0.514112,0.492388
3,HOBBIES_1_004_CA_1_evaluation,F1,1.570784,1.640950,1.0000,1.420823,1.562155
4,HOBBIES_1_005_CA_1_evaluation,F1,0.839118,0.883593,0.9640,1.079060,0.969789


In [461]:
print(train_data.shape,data_true.shape)
print(test_data.shape, data_true.shape)

(853720, 7) (853720, 3)
(853720, 7) (853720, 3)


In [462]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

#cv = KFold(n_splits=4, shuffle=True, random_state=42)

'''
store_ids = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX2', 'TX3', 'WI_1', 'WI_2', 'WI_3']
for i, store_id in enumerate(store_ids):
    train_df = train_data[train_data['id'].str.contains(store_id)].iloc[:, 2:]
    test_tmp = data_true[data_true['id'].str.contains(store_id)]
    test_df = test_tmp['value'].values
    print(train_data.shape, data_true.shape)
    cv = KFold(n_splits=4, shuffle=True, random_state=42)
    for tr_idx, val_idx in cv.split(train_df, test_df):
        print(len(tr_idx))
        X_tr = train_df.iloc[tr_idx].values
        y_tr = test_df[tr_idx]
        X_val = train_df.iloc[val_idx]
        rg = Ridge()
        rg.fit(X_tr, y_tr)
        preds_tmp = rg.predict(X_val)
        print(preds_tmp)
'''

"\nstore_ids = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX2', 'TX3', 'WI_1', 'WI_2', 'WI_3']\nfor i, store_id in enumerate(store_ids):\n    train_df = train_data[train_data['id'].str.contains(store_id)].iloc[:, 2:]\n    test_tmp = data_true[data_true['id'].str.contains(store_id)]\n    test_df = test_tmp['value'].values\n    print(train_data.shape, data_true.shape)\n    cv = KFold(n_splits=4, shuffle=True, random_state=42)\n    for tr_idx, val_idx in cv.split(train_df, test_df):\n        print(len(tr_idx))\n        X_tr = train_df.iloc[tr_idx].values\n        y_tr = test_df[tr_idx]\n        X_val = train_df.iloc[val_idx]\n        rg = Ridge()\n        rg.fit(X_tr, y_tr)\n        preds_tmp = rg.predict(X_val)\n        print(preds_tmp)\n"

In [463]:
train_df = train_data.iloc[:, 2:]
train_y_df = data_true['value'].values
test_df = test_data.iloc[:, 2:]

cv = KFold(n_splits=4, shuffle=True, random_state=42)

preds = []
preds_test = []
va_idxes = []
for tr_idx, val_idx in cv.split(train_df, test_df):
    #print(len(tr_idx))
    X_tr = train_df.iloc[tr_idx].values
    y_tr = train_y_df[tr_idx]
    X_val = train_df.iloc[val_idx]
    rg = Ridge(alpha=1)
    rg.fit(X_tr, y_tr)
    print(rg.coef_)
    print(rg.intercept_)
    pred_tmp = rg.predict(X_val)
    pred_test = rg.predict(test_df.values)
    preds.append(pred_tmp)
    va_idxes.append(val_idx)
    preds_test.append(pred_test)
    #print(preds_tmp)

va_idxes = np.concatenate(va_idxes)
preds = np.concatenate(preds, axis=0)
order = np.argsort(va_idxes)
pred_train = preds[order]
preds_test = np.mean(preds_test, axis=0)

#pred_train

[-0.09780946  0.39890798  0.08168893  0.53303509  0.12096158]
-0.02120338362089913
[-0.04222305  0.36040996  0.09568652  0.52816025  0.09249111]
-0.017858728293939308
[-0.02939869  0.3343806   0.11814758  0.51705106  0.0953431 ]
-0.017507979817982244
[-0.0325052   0.34191761  0.09988114  0.49825208  0.12552371]
-0.016656284488987483


In [464]:
sub_tmp = pd.read_csv('../data/input/m5-forecasting-accuracy/sample_submission.csv')
sub_tmp = sub_tmp[sub_tmp.id.str.endswith('evaluation')]
sub_tmp.rename(columns={f'F{i+1}': i+1 for i in range(28)}, inplace=True)

In [465]:
sub_tmp = pd.melt(sub_tmp, id_vars=['id'])
sub_tmp_val = sub_tmp
sub_tmp_te = sub_tmp

In [466]:
sub_tmp_val['value'] = pred_train
sub_tmp_te['value'] = preds_test
#sub_tmp.head()

In [467]:
sub_tmp_val = pd.pivot(sub_tmp_val, index='id', columns='variable', values='value')
sub_tmp_te = pd.pivot(sub_tmp_te, index='id', columns='variable', values='value')

In [468]:
sub_merge = pd.read_csv('../data/input/m5-forecasting-accuracy/sample_submission.csv')
sub_merge = sub_merge.iloc[30490:, 0]
sub_merge_val = sub_merge
sub_merge_te = sub_merge
sub_tmp_val = pd.merge(sub_merge_val, sub_tmp_val, how='left', on='id')
sub_tmp_te = pd.merge(sub_merge_te, sub_tmp_te, how='left', on='id')

In [469]:
submission = pd.read_csv('../data/input/m5-forecasting-accuracy/sample_submission.csv')
submission_val = submission
submission_te = submission
submission_val.iloc[30490:, 1:] = sub_tmp_val.iloc[:, 1:].values
submission_te.iloc[30490:, 1:] = sub_tmp_te.iloc[:, 1:].values

In [470]:
submission_val.to_csv('../data/output/submission_stacking_all_val.csv', index=False)
submission_te.to_csv('../data/output/submission_stacking_all_te.csv', index=False)